In [ ]:
!pip install transformers -qq
!pip install sentencepiece -qq
!pip install numpy requests nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 14.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Load Dataset

In [ ]:
import random
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw

from nlpaug.util import Action

In [ ]:
alpha_common_error = 0.10
alpha_common_error_char = 0.05
aug1_OCR = nac.OcrAug(aug_word_p=alpha_common_error)
aug2_Rins = nac.RandomCharAug(action="insert", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug3_Rsub = nac.RandomCharAug(action="substitute", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug4_Rswa = nac.RandomCharAug(action="swap", aug_word_p=alpha_common_error,aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char) #
aug5_Rdel = nac.RandomCharAug(action="delete", aug_word_p=alpha_common_error, aug_char_min=1, aug_char_max=1, aug_char_p=alpha_common_error_char)
aug6_Kb = nac.KeyboardAug(aug_word_p=alpha_common_error)
aug7_Split = naw.SplitAug(aug_p=alpha_common_error)

In [ ]:
random.sample(range(0, 7), 2 - 1)

[6]

In [ ]:
def text2augment(text, m):
    output = [text, ]

    temp = random.sample(range(0, 7), m - 1)

    if 0 in temp:
        output.append(*aug1_OCR.augment(text))
    if 1 in temp:
        output.append(*aug2_Rins.augment(text))
    if 2 in temp:
        output.append(*aug3_Rsub.augment(text))
    if 3 in temp:
        output.append(*aug4_Rswa.augment(text))
    if 4 in temp:
        output.append(*aug5_Rdel.augment(text))
    if 5 in temp:
        output.append(*aug6_Kb.augment(text))
    if 6 in temp:
        output.append(*aug7_Split.augment(text))

    return output

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def to_df(x, y):
    d = {"text": x, "label": y}
    return pd.DataFrame(d)

def split_3(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = _df["text"].copy()
    y = _df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y)
    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
from itertools import chain

In [ ]:
def aug_replicate(y_labels):
    return list(chain(* [[y]*(2 if y == 0 else 4) for y in y_labels] ))

def aug_text(x_text, y_labels):
    x_text = [ text2augment(x, 2 if y == 0 else 4) for x, y in zip(x_text, y_labels)]
    return pd.Series(list(chain(*x_text)), index=None)

def split_3_aug(df, test_size=0.2, valid_size=0.2):
    _df = df.copy().sample(frac=1).reset_index()
    _df = _df[["text", "label"]]

    x = _df["text"].copy()
    y = _df["label"].copy()
    #split train-test
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, stratify=y)
    # augment
    # x_test = aug_text(x_test, y_test)
    # y_test = aug_replicate(y_test)
    # split train-valid
    x, y = x_train, y_train
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=valid_size, stratify=y)
    # augment
    x_train = aug_text(x_train, y_train)
    y_train = aug_replicate(y_train)
    x_valid = aug_text(x_valid, y_valid)
    y_valid = aug_replicate(y_valid)

    print(x_valid.shape)
    print("DONE")
    print(len(y_valid))

    print(x_train.shape)
    print("DONE")
    print(len(y_train))

    return to_df(x_train, y_train), to_df(x_valid, y_valid), to_df(x_test, y_test)

In [ ]:
import numpy as np
import pandas as pd

tname_data = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/hsd_merge_cleaned_lowered"
data = pd.read_csv(f"{tname_data}.csv")

train, valid, test = split_3_aug(data)

X_train = train['text']
y_train = train['label']

X_valid = valid['text']
y_valid = valid['label']

X_test = test['text']
y_test = test['label']

(5772,)
DONE
5772
(23082,)
DONE
23082


In [ ]:
# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_train.csv"
# train.to_csv(temp,  sep=",", index=False)

# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_valid.csv"
# valid.to_csv(temp,  sep=",", index=False)

# temp = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/final/augmented/hsd_merge_aug_test.csv"
# test.to_csv(temp,  sep=",", index=False)

In [ ]:
data[data["label"] == 1]

,text,label
3,muy despreciados siiii pero todos vestidos de ...,1
4,marica explicame porque a veces no te entiendo...,1
8,jajajajaja te jodes maricon,1
12,rovira lleva siendo subnormal desde 1980,1
15,dale a me gusta en mi página de facebook y rec...,1
...,...,...
15778,marinovic hay alguna mujer del movimiento femi...,1
15782,leí tu descripción ahora entiendo todo el daño...,1
15816,🤮🤮🤮🤮una 💩 de debate solo orrego se presta para...,1
15822,los delincuentes de la primers linea queman ig...,1


In [ ]:
train[train["label"] == 1]

,text,label
14,me acojona ver a la derecha rancia pidiendo má...,1
15,me acosjona ver a la derecha rancia pidiendo m...,1
16,me acojona ver a la derecha rancia pidiendo má...,1
17,me acojona ver a la derecha rancia pidiendo má...,1
34,mcsexy madre mía menuda subnormal,1
...,...,...
23011,me da mucha rabia porque todo el mu ndo hace 4...,1
23042,en general los d la cup son muy lerdos pero lo...,1
23043,en general los d la cup son muy lerdos pero lo...,1
23044,en general los d la cup son muy elrdos pero lo...,1


In [ ]:
valid[valid["label"] == 1]

,text,label
6,de los mismos de reforma tributaria procrecimi...,1
7,de los mismos de kefukma tributaria procrecimi...,1
8,de los mismos de reforma tributaria procrecimi...,1
9,de los mismos de reforma tributaria procrecimi...,1
10,vete a la mierda malparido y si no es por plat...,1
...,...,...
5733,co lina lo que vale callampa es la palabra de ...,1
5744,va por ti después de las porras después del 15...,1
5745,va por ti después de las porras después del 15...,1
5746,va por ti de@spués de las Lporras después del ...,1


In [ ]:
test[test["label"] == 1]

,text,label
10644,cada vez q una feminazi me insulta una de 18 m...,1
5113,el cojo y el que defrauda por algo estás en cu...,1
13922,estoy harto me cansé de perú no puedo seguir v...,1
13343,beatle tomasroncero son mucho fachas madridistas,1
9596,antes de follar con alguna loca del coño haceo...,1
...,...,...
15749,mongolo lama manu carroña y bobo gómez alias g...,1
12498,pero mira que hdp estos rojos de podemos hacie...,1
9415,estos basuras como le tienen miedo a los carpe...,1
9168,eres un asco señor abad un asco tu y la puta l...,1


In [ ]:

# import numpy as np
# import pandas as pd

# tname_data = "/content/drive/MyDrive/TP1-AntiDiscriminator/datasets/original/dataset_9834_cleaned_lowered"
# data = pd.read_csv(f"{tname_data}.csv")
# # test = pd.read_csv(f"{tname_data}_test.csv")

# data = data.sample(frac=1).reset_index()
# data = data[["text", "label"]]

# train = data[: int(len(data) * 0.7)]
# valid = data[int(len(data) * 0.7) : int(len(data) * 0.9)]
# test = data[int(len(data) * 0.9) :]

# X_train = train['text']
# y_train = train['label']

# X_valid = valid['text']
# y_valid = valid['label']

# X_test = test['text']
# y_test = test['label']

In [ ]:

# import numpy as np
# import pandas as pd

# tname_data = "hsd_merged_AUG"

# data = pd.read_csv(f"{tname_data}_no_test.csv")
# test = pd.read_csv(f"{tname_data}_test.csv")


# train = data[: int(len(data) * 0.9)]
# valid = data[int(len(data) * 0.9) :]

# X_train = train['text']
# y_train = train['label']

# X_valid = valid['text']
# y_valid = valid['label']

# X_test = test['text']
# y_test = test['label']


## Set Cuda

In [ ]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.cuda.is_available()

True

# Extract feature by using BETO

In [ ]:
train_sentences

['mujer trató de quemarse a lo bonzo frente al centro de justicia',
 'mu jer trató de quemarse a lo b onzo frente al centro de justicia',
 'te ha quedado un poco de commercial yanki barato eh jajaja',
 'te ha quedado un poco de commercial yanki barato eh jajaja',
 'cualquiera que gane será lo mismo ganan los zurdos pero si gana orrego lo hará con los votos de la derecha débil y lo que tenga que pasar pasará igual nuestro voto debiera ser nulo acostumbremos a votar por los nuestros nunca por el adversario',
 'cualquiera que gane será lo mi8mu 9anan los zurdos pero si gana orrego lo hará con los votos de la derecha débil y lo que tenga que pasar pasará igual nuestro voto debiera ser nulo acostumbremos a votar por los nuestros nunca puk el advek8ari0',
 'aquí tienes otros datos por si te interesan también voto independentista 2 042 919 voto no independentista 2 234 023',
 'aquí tienes utkos datos por si te interesan también voto independentista 2 042 919 voto no independentista 2 234 oz3'

In [ ]:
import pandas as pd
from glob import glob

train_sentences = list(train['text'].values)
train_labels = list(train['label'].values)

valid_sentences = list(valid['text'].values)
valid_labels = list(valid['label'].values)

test_sentences = list(test['text'].values)
test_labels = list(test['label'].values)

Load tokenizer of BETO

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

In [ ]:
tokenizer.vocab_size

31002

In [ ]:
#choose max_length for beto model based on the input length

max_length = 0
list_len=[]
for sentence in train_sentences:
    length = len(tokenizer.tokenize(sentence))
    list_len.append(length)

from collections import Counter
Counter(list_len).most_common(100)

[(24, 717),
 (22, 682),
 (19, 682),
 (21, 681),
 (23, 678),
 (25, 666),
 (18, 657),
 (20, 657),
 (17, 654),
 (12, 648),
 (26, 643),
 (13, 634),
 (27, 633),
 (15, 619),
 (16, 612),
 (11, 606),
 (14, 604),
 (10, 578),
 (28, 577),
 (29, 568),
 (9, 515),
 (31, 493),
 (30, 483),
 (32, 458),
 (33, 438),
 (8, 422),
 (34, 390),
 (36, 360),
 (35, 342),
 (7, 328),
 (37, 278),
 (6, 251),
 (39, 249),
 (38, 247),
 (42, 228),
 (41, 213),
 (40, 211),
 (43, 200),
 (44, 198),
 (46, 191),
 (49, 174),
 (50, 173),
 (47, 172),
 (51, 168),
 (54, 167),
 (53, 164),
 (5, 163),
 (57, 161),
 (45, 160),
 (48, 155),
 (52, 147),
 (58, 144),
 (55, 137),
 (56, 136),
 (61, 129),
 (59, 122),
 (60, 120),
 (64, 109),
 (63, 101),
 (65, 93),
 (62, 89),
 (66, 88),
 (67, 87),
 (4, 84),
 (68, 67),
 (70, 55),
 (69, 55),
 (71, 45),
 (73, 40),
 (72, 37),
 (74, 33),
 (75, 31),
 (3, 30),
 (78, 26),
 (76, 22),
 (77, 19),
 (2, 13),
 (82, 12),
 (80, 11),
 (79, 7),
 (81, 6),
 (86, 5),
 (1, 4),
 (83, 4),
 (93, 3),
 (84, 3),
 (89, 3),
 

In [ ]:
# Encode train label

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(train_labels)
encoded_train_labels = le.transform(train_labels)
encoded_valid_labels = le.transform(valid_labels)
encoded_test_labels = le.transform(test_labels)

In [ ]:
# Tokens IDs tensor

def encoder_generator(sentences,labels):

    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):

        sent_index.append(index)

        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=50,
                                             padding='max_length',
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0).cuda()
    attention_masks = torch.cat(attention_masks,dim=0).cuda()
    labels = torch.tensor(labels).cuda()
    sent_index = torch.tensor(sent_index).cuda()

    return sent_index,input_ids,attention_masks,labels

train_sent_index,train_input_ids,train_attention_masks,train_encoded_label_tensors = encoder_generator(train_sentences,encoded_train_labels)
valid_sent_index,valid_input_ids,valid_attention_masks,valid_encoded_label_tensors = encoder_generator(valid_sentences,encoded_valid_labels)
print('Original: ', train_sentences[0])
print('Token IDs:', train_input_ids[0])

Original:  mujer trató de quemarse a lo bonzo frente al centro de justicia
Token IDs: tensor([    4,  1729, 12138,  1008, 18323,  1150,  1013,  1114,  4178,  1558,
         2999,  1091,  3040,  1008,  4815,     5,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1],
       device='cuda:0')


In [ ]:
# Connvert train, dev input by using TensorDataset

from torch.utils.data import TensorDataset,random_split

train_dataset = TensorDataset(train_input_ids,train_attention_masks,train_encoded_label_tensors)
valid_dataset = TensorDataset(valid_input_ids,valid_attention_masks,valid_encoded_label_tensors)

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(valid_dataset)))

train data samples is 23082
valid data samples is 5772


In [ ]:
# Set cuda by using device

from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(valid_dataset,
                              sampler=RandomSampler(valid_dataset),
                              batch_size=bs)

Load model BETO

In [ ]:
from transformers import AutoModel

beto = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')
beto = beto.to(device)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

# Build CNN

In [ ]:
# Build class CNN

import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel

class CNN(nn.Module):
    def __init__(self, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):

        super().__init__()

        self.fc_input = nn.Linear(embedding_dim,embedding_dim)

        self.conv_0 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[0])

        self.conv_1 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[1])

        self.conv_2 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[2])

        self.conv_3 = nn.Conv1d(in_channels = embedding_dim,
                                out_channels = n_filters,
                                kernel_size = filter_sizes[3])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, encoded):

        #embedded = [batch size, sent len, emb dim]
        embedded = self.fc_input(encoded)
        #print(embedded.shape)

        embedded = embedded.permute(0, 2, 1)
        #print(embedded.shape)

        #embedded = [batch size, emb dim, sent len]

        conved_0 = F.relu(self.conv_0(embedded))
        conved_1 = F.relu(self.conv_1(embedded))
        conved_2 = F.relu(self.conv_2(embedded))
        conved_3 = F.relu(self.conv_3(embedded))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        pooled_3 = F.max_pool1d(conved_3, conved_3.shape[2]).squeeze(2)

        #pooled_n = [batch size, n_fibatlters]

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2, pooled_3), dim = 1).cuda())

        #cat = [batch size, n_filters * len(filter_sizes)]

        result =  self.fc(cat)

        #print(result.shape)

        return result

In [ ]:
# Hyperparameters

EMBEDDING_DIM = 768
N_FILTERS = 32
FILTER_SIZES = [1,2,3,5]
OUTPUT_DIM = len(le.classes_)
DROPOUT = 0.2
PAD_IDX = tokenizer.pad_token_id

cnn = CNN(EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
cnn = cnn.to(device)

In [ ]:
cnn

CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
0.00002 == 2e-5

True

In [ ]:
# Optimizer and criterion

import torch.optim as optim

model_parameters = list(beto.parameters())+list(cnn.parameters())

optimizer = optim.Adam(model_parameters,lr=2e-5,eps=1e-8)
# optimizer = optim.Adam(model_parameters,lr=1e-3,eps=1e-8)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [ ]:
# Calculate accuracy per batch during train

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

In [ ]:
# Def for training

from tqdm import tqdm

def train():

    epoch_loss = 0
    epoch_acc = 0

    beto.train()
    cnn.train()

    for batch in tqdm(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        embedded = beto(b_input_ids,b_input_mask)[0]

        predictions = cnn(embedded)

        loss = criterion(predictions, b_labels)

        acc = categorical_accuracy(predictions, b_labels)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(train_data_loader), epoch_acc / len(train_data_loader)

In [ ]:
# Class for predict label

import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
# Evaluate loss, acc  and f1-macro

from sklearn.metrics import classification_report,accuracy_score,f1_score
def eval():
    epoch_loss = 0

    total_predictions = []
    total_true = []

    all_true_labels = []
    all_pred_labels = []

    beto.eval()
    cnn.eval()

    with torch.no_grad():

        for batch in tqdm(valid_data_loader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            embedded = beto(b_input_ids,b_input_mask)[0]
            predictions = cnn(embedded)

            loss = criterion(predictions, b_labels)
            epoch_loss += loss.item()

            predictions = predictions.detach().cpu().numpy()

            label_ids = b_labels.to('cpu').numpy()

            pred,true = predictions_labels(predictions,label_ids)

            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')

    avg_val_loss = epoch_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))

    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
# Time for training

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# # Set device and gpu

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0)

# beto.cuda()

# Training

In [ ]:
tname = "/content/beto_cnn_model_home"

In [ ]:
# epochs = 20
# epochs = 10
epochs = 5

best_macro_f1 = float('0')

for epoch in range(epochs):

    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = eval()
    end_time = time.time()


    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(beto,f'{tname}_part1_train.pt')
        torch.save(cnn,f'{tname}_part2_train.pt')
        print("MODEL SAVED")

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. acc: {valid_acc*100:.2f}%')
    print(f'\t Val. F1: {macro_f1*100:.2f}%')
    print('=============Epoch Ended==============\n')

100%|██████████| 46/46 [00:14<00:00,  3.10it/s]


              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4472
           1       0.58      0.63      0.60      1300

    accuracy                           0.81      5772
   macro avg       0.74      0.75      0.74      5772
weighted avg       0.82      0.81      0.82      5772

accuracy = 0.81
MODEL SAVED
Epoch: 01 | Epoch Time: 3m 9s
	Train Loss: 0.436 | Train acc: 80.32%
	 Val. Loss: 0.412 |  Val. acc: 81.46%
	 Val. F1: 74.19%
=============Epoch Ended==============



100%|██████████| 46/46 [00:14<00:00,  3.09it/s]


              precision    recall  f1-score   support

           0       0.96      0.82      0.89      5104
           1       0.35      0.75      0.48       668

    accuracy                           0.81      5772
   macro avg       0.66      0.78      0.68      5772
weighted avg       0.89      0.81      0.84      5772

accuracy = 0.81
Epoch: 02 | Epoch Time: 3m 8s
	Train Loss: 0.235 | Train acc: 90.44%
	 Val. Loss: 0.608 |  Val. acc: 81.29%
	 Val. F1: 68.29%
=============Epoch Ended==============



100%|██████████| 46/46 [00:14<00:00,  3.10it/s]


              precision    recall  f1-score   support

           0       0.95      0.84      0.89      4961
           1       0.42      0.73      0.53       811

    accuracy                           0.82      5772
   macro avg       0.69      0.78      0.71      5772
weighted avg       0.88      0.82      0.84      5772

accuracy = 0.82
Epoch: 03 | Epoch Time: 3m 8s
	Train Loss: 0.090 | Train acc: 96.67%
	 Val. Loss: 0.761 |  Val. acc: 82.10%
	 Val. F1: 71.18%
=============Epoch Ended==============



100%|██████████| 46/46 [00:14<00:00,  3.09it/s]


              precision    recall  f1-score   support

           0       0.88      0.88      0.88      4357
           1       0.63      0.63      0.63      1415

    accuracy                           0.82      5772
   macro avg       0.76      0.76      0.76      5772
weighted avg       0.82      0.82      0.82      5772

accuracy = 0.82
MODEL SAVED
Epoch: 04 | Epoch Time: 3m 8s
	Train Loss: 0.040 | Train acc: 98.61%
	 Val. Loss: 0.715 |  Val. acc: 82.07%
	 Val. F1: 75.73%
=============Epoch Ended==============



100%|██████████| 46/46 [00:14<00:00,  3.10it/s]

              precision    recall  f1-score   support

           0       0.96      0.82      0.89      5105
           1       0.35      0.74      0.48       667

    accuracy                           0.81      5772
   macro avg       0.66      0.78      0.68      5772
weighted avg       0.89      0.81      0.84      5772

accuracy = 0.81
Epoch: 05 | Epoch Time: 3m 8s
	Train Loss: 0.023 | Train acc: 99.28%
	 Val. Loss: 0.971 |  Val. acc: 81.17%
	 Val. F1: 68.07%
=============Epoch Ended==============



In [ ]:
# Save BETO and CNN

torch.save(beto, tname + '_module_part1.pt')
torch.save(cnn,  tname + '_module_part2.pt')

In [ ]:
# # Save BETO and CNN
# tname = "/content/drive/MyDrive/TP1-AntiDiscriminator/weights/beto_cnn"
# torch.save(beto, tname + '_part1_train.pt')
# torch.save(cnn,  tname + '_part2_train.pt')

## EVALUATING

In [ ]:
# Load phobert and cnn

import torch
beto = torch.load(f'{tname}_part1_train.pt')
cnn = torch.load(f'{tname}_part2_train.pt')
beto.eval()
cnn.eval()


CNN(
  (fc_input): Linear(in_features=768, out_features=768, bias=True)
  (conv_0): Conv1d(768, 32, kernel_size=(1,), stride=(1,))
  (conv_1): Conv1d(768, 32, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(768, 32, kernel_size=(3,), stride=(1,))
  (conv_3): Conv1d(768, 32, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

Predict label from true label

In [ ]:
test_sent_index, test_input_ids, test_attention_masks, test_encoded_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
test_dataset = TensorDataset(test_input_ids,test_attention_masks,test_encoded_label_tensors)

test_data_loader = DataLoader(test_dataset,
                              sampler=RandomSampler(test_dataset),
                              batch_size=bs)

all_pred_labels = []
all_true_labels = []

with torch.no_grad():
  for batch in tqdm(test_data_loader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    embedded = beto(b_input_ids,b_input_mask)[0]
    predictions = cnn(embedded)


    predictions = predictions.detach().cpu().numpy()

    label_ids = b_labels.to('cpu').numpy()

    pred, true = predictions_labels(predictions, label_ids)

    all_pred_labels.extend(pred)
    all_true_labels.extend(true)

100%|██████████| 25/25 [00:08<00:00,  3.08it/s]


In [ ]:
# The final score in the test set (classification report)

print(classification_report(all_pred_labels,all_true_labels, digits = 4))

              precision    recall  f1-score   support

           0     0.9190    0.9268    0.9229      2704
           1     0.5500    0.5227    0.5360       463

    accuracy                         0.8677      3167
   macro avg     0.7345    0.7247    0.7294      3167
weighted avg     0.8650    0.8677    0.8663      3167



In [ ]:
# Confusion matrix in the test set

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(all_true_labels, all_pred_labels)
cm

array([[2506,  221],
       [ 198,  242]])